# Exploring Retrosheet
This notebook contains some routines for reading in [Retrosheet](https://retrosheet.org) data and putting it into a useable format for analysis.

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import numpy as np
import pandas as pd
import datetime, copy, imp
import time
import os
import re
import matplotlib.pyplot as plt

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
tqdm.pandas()

import sys
sys.path.insert(0, '../util/')

%aimport data_cache
from data_cache import CacheResult

In [4]:
cachePathStr = '/Users/gmessier/data/baseball/cache/'
dataPathStr = '/Users/gmessier/data/baseball'
tgtSeasons = [ 2016, 2017 ]  # The seasons that define our player population.
startSeason = 1992  # Player event start season.
endSeason = 2019 # Player event end season.
topPlayerFrc = 0.1 # Fraction of players to select as top performers.

## File Names
- Create arrays that contain all the roster and event files for a particular season.

In [ ]:
dataPathStr = '/Users/gmessier/data/baseball'


## Player Rosters
Read all the rosters into a single 2D array.

In [5]:
season = 2018
teamFile = 'ANA2018.ROS'
fileStr = f'{dataPathStr}/{season}/{teamFile}'

In [11]:
ros = np.genfromtxt(fileStr,delimiter=',',dtype='str')

## Target Player Roster
- The player population is every non-pitcher who was playing in the target seasons.

In [4]:
rosterFiles = [] 
for tgtSeason in tgtSeasons:
    pathStr = dataPathStr + '{}/'.format(tgtSeason)
    allFiles = os.listdir(pathStr)
    fileRegex = re.compile('ROS')
    rosterFiles += [ '{}/{}'.format(tgtSeason,x) for x in allFiles if fileRegex.search(x) ]

In [5]:
@CacheResult
def ProcessRosters(fileNames):
    rowList = []
    for fileName in tqdm(fileNames):    
        file = open(dataPathStr + fileName)        
        for line in file:
            flds = line.split(',')
            row = {
                'PlayerId': flds[0],
                'LastName': flds[1],
                'FirstName': flds[2],
                'Posn': flds[-1][0:-1],
                'Team': flds[-2]
            }
            rowList.append(row)
            
        file.close()
        
    tbl = pd.DataFrame.from_dict(rowList)
    
    # Since we mainly care about position and not team, arbitrarily keep one duplicated record.
    tbl = tbl.loc[~(tbl.PlayerId.duplicated())] 
            
    return tbl

In [6]:
rstr = ProcessRosters(rosterFiles,path=cachePathStr)

In [7]:
# Do not include pitchers in the assessment of offsensive capability.
offPlayers = rstr[rstr.Posn != 'P'].PlayerId

## Player Transactions

In [8]:
tranFileStr = '~/data/baseball/tran/tran.txt'

In [9]:
headers = [
    'PriDate', 'Time', 'ApxInd', 'SecDate', 'ApxIndSec', 'TrId',
    'Player', 'Type', 'FromTeam', 'FromLeague', 'ToTeam',
    'ToLeague', 'DftType', 'DftRnd', 'PickNo', 'Info']

types = {
    'PriDate': 'object', 'Time': 'object', 'ApxInd': 'object', 
    'SecDate': 'object', 'ApxIndSec': 'object', 'TrId': 'object',
    'Player': 'object', 'Type': 'object', 'FromTeam': 'object', 
    'FromLeague': 'object', 'ToTeam': 'object',
    'ToLeague': 'object', 'DftType': 'object', 'DftRnd': 'object', 
    'PickNo': 'object', 'Info': 'object'}

In [10]:
trnRaw = pd.read_csv(tranFileStr,sep=',',names=headers,dtype=types)

In [11]:
tranEventType = { 
    'F': 'FreeAgent', 'R': 'Released', 'D': 'Draft', 'T': 'Trade', 'W': 'Waivers', 'C': 'CndDeal', 'P': 'Purchase',
    'U': 'Unknown', 'X': 'ExpansionPick', 'M': 'MinorLeague', 'L': 'Loan', 'J': 'Jump', 'A': 'Assigned', 'V': 'LeagueCtrl',
    'Z': 'Retired', 'p': 'Purchase', np.nan: np.nan }

def TransactionTimeline(tbl):
    
    dates = pd.to_datetime(pd.DataFrame({ 
        'year': tbl.PriDate.str[:4].astype('int'), 
        'month': tbl.PriDate.str[4:6].astype('int').combine(1,max), 
        'day': tbl.PriDate.str[6:8].astype('int').combine(1,max),
        'minute': [ 1 ]*len(tbl.index)}))
    
    return pd.DataFrame({
        'Date': dates, 
        'Event': [ tranEventType[t] for t in tbl.Type.str[0] ] 
        }).sort_values('Date')

In [12]:
@CacheResult
def GenTransactionTimelines(trn):
    return trnRaw.groupby('Player').progress_apply(TransactionTimeline)

In [13]:
trn = GenTransactionTimelines(trnRaw,path=cachePathStr)
trn = trn[trn.index.get_level_values(0).isin(offPlayers)]

## Process Event Files

In [14]:
seasons = range(startSeason,endSeason+1)

In [15]:
dataPathStr = '/Users/gmessier/data/baseball/'
fileRegex = re.compile('EV(A|N)')
eventFiles = []
for season in seasons:
    pathStr = '{}{}/'.format(dataPathStr,season)
    allFiles = os.listdir(pathStr)
    for x in allFiles:
        if fileRegex.search(x):
            eventFiles.append('{}{}'.format(pathStr,x))

In [16]:
hitRegex = re.compile("^(S\d|D\d|T\d|DGR|HR|HP)"); # For now, includes being hit by a pitch.
singleRegex = re.compile("^S|^HP");
doubleRegex = re.compile("^D");
tripleRegex = re.compile("^T");
homerunRegex = re.compile("^H");
rbiRegex = re.compile("-H");
bbRegex = re.compile("^(?:IW|W(?:[^P]|$))")

In [17]:
def ProcessOffensiveEvents(eventFileStr):

    fp = open(eventFileStr)
    rowList = []

    gameStr = ''
    topBottom = [ 'Top', 'Bottom' ]

    for line in fp:
        row = {}
        flds = line.split(',')

        if flds[0] == 'id':
            gameStr = flds[1][0:-1]
            dateStr = gameStr[3:7] + '-' + gameStr[7:9] + '-' + gameStr[9:11]
            if gameStr[-1] == '0':
                curTime = pd.to_datetime(dateStr + ' 12:00')
            elif gameStr[-1] == '1':
                curTime = pd.to_datetime(dateStr + ' 15:01')
            elif gameStr[-1] == '2':
                curTime = pd.to_datetime(dateStr + ' 18:02')
            else:
                sys.exit("ERROR: Can't handle more than a triple header.")

        if flds[0] == 'start':

            entry = { 'Date': curTime, 'PlayerId': flds[1], 'Event': 'Start', 'Rbi': 0 }
            rowList.append(entry)
            
        if flds[0] == 'play':
            curTime += pd.Timedelta('1 min')
            result = flds[6][0:-1]
                            
            if hitRegex.search(result):
                entry = { 'Date': curTime, 'PlayerId': flds[3], 'Event': '', 'Rbi': len(rbiRegex.findall(result)) }

                if singleRegex.search(result):
                    entry['Event'] = '1B'
                elif doubleRegex.search(result):
                    entry['Event'] = '2B'
                elif tripleRegex.search(result):
                    entry['Event'] = '3B'
                else:
                    entry['Event'] = 'HR'

                rowList.append(entry)
                    
            if bbRegex.search(result):
                curTime += pd.Timedelta('1 min')
                entry = { 'Date': curTime, 'PlayerId': flds[3], 'Event': 'BB', 'Rbi': len(rbiRegex.findall(result)) }
                rowList.append(entry)
            
    
    return pd.DataFrame.from_dict(rowList)    
        

In [18]:
#tt = ProcessOffensiveEvents('/Users/gmessier/data/baseball/2017/2017ANA.EVA')

In [19]:
@CacheResult
def ProcessEventFiles(files):
    tbl = pd.DataFrame()
    for file in tqdm(files):
        tbl = pd.concat([ tbl, ProcessOffensiveEvents(file) ],ignore_index=True)
    return tbl    

In [20]:
evnt = ProcessEventFiles(eventFiles,path=cachePathStr)
evnt = evnt[evnt.PlayerId.isin(offPlayers)]

## Offsensive Performance

In [21]:
yhooPts = {
    '1B': 2.6, '2B': 5.2, '3B': 7.8, 'HR': 10.4, 'R': 1.9, 'RBI': 1.9, 'BB': 2.6, 'SB': 4.2, 'HBP': 2.6
}

In [22]:
def TotalGamePoints(tbl):
    points = 0
    
    points += sum(tbl.Event == '1B') * yhooPts['1B']
    points += sum(tbl.Event == '2B') * yhooPts['2B']
    points += sum(tbl.Event == '3B') * yhooPts['3B']
    points += sum(tbl.Event == 'HR') * yhooPts['HR']
    points += sum(tbl.Event == 'BB') * yhooPts['BB']

    points += tbl.Rbi.sum() * yhooPts['RBI']

    return points

In [23]:
def PlayerGamePointTotals(tbl):
    pts = tbl.groupby(tbl.Date.dt.date).apply(CalcGamePoints)
    return pd.DataFrame({
        'Date': pd.to_datetime(pts.index)+pd.Timedelta('23 hours'),
        'Points': pts
        })

In [24]:
@CacheResult
def CalcPlayerPoints(evnt):
    return evnt.groupby('PlayerId').progress_apply(PlayerGamePointTotals)

In [25]:
# Determines the total Yahoo points scored by each player on each of their game dates.
playerPts = CalcPlayerPoints(evnt,path=cachePathStr)

In [26]:
def CalcPlayerSeasonTotals(tbl):
    return tbl.groupby(tbl.Date.dt.year).apply(lambda x: sum(x.Points))

In [27]:
@CacheResult
def CalcSeasonTotals(pts):
    return pts.groupby(level=0).progress_apply(CalcPlayerSeasonTotals)

In [28]:
# Determines the total points scored by each player in a season.
seaTtls = CalcSeasonTotals(playerPts,path=cachePathStr)

In [29]:
def CalcTopPlayerCutoff(tbl,cutoffFrc):
    years = tbl.index.get_level_values(1).unique().sort_values()
    cutoffs = []
    for year in years:
        nPlayers = len(tbl.loc[:,year])
        cutoffs.append(tbl.loc[:,year].sort_values().iloc[int(nPlayers*(1-cutoffFrc))])
    return pd.Series(cutoffs,index=years)

In [30]:
# The cutoffs in each season to be in the top topPlayerFrc of players.
topThsh = CalcTopPlayerCutoff(seaTtls,topPlayerFrc)

In [31]:
def FindTopPlayerEvents(tbl,thresholds):
    years = tbl.Date.dt.year.unique()
    eventDates = []
    for year in years:
        seasn = tbl[tbl.Date.dt.year == year]
        cumPts = seasn.Points.cumsum()
        if cumPts.iloc[-1] > thresholds[year]:
            eventDates.append(seasn.loc[cumPts > thresholds[year]].Date.min())
    return pd.DataFrame({'Date': eventDates, 'Event': [ 'TopPlayer' ]*len(eventDates)})

In [32]:
@CacheResult
def FindTopPlayers(playerPts,thresholds):
    return playerPts.groupby(level=0).progress_apply(FindTopPlayerEvents,thresholds=topThsh)

In [33]:
# Produce a time stamped table of when/if a player breaks into the top topPlayerFrc
evntTop = FindTopPlayers(playerPts,topThsh,path=cachePathStr)

## Translate Baseball Events to Medical Events

In [34]:
def PlayerMajorEvents(tbl):
    return tbl.loc[tbl.Event.isin(['Start','HR'])][['Date','Event']]

In [35]:
#PlayerMajorEvents(evnt.loc[evnt.PlayerId == 'abrej003'])

In [36]:
@CacheResult
def MajorEvents(evnt):
    return evnt.groupby('PlayerId').progress_apply(PlayerMajorEvents)

In [37]:
evntMjr = MajorEvents(evnt,path=cachePathStr)

In [38]:
tbl = pd.concat([ trn, evntMjr, evntTop ])
tbl = tbl.groupby(level=0).progress_apply(lambda x: x.sort_values(by=['Date'],ignore_index=True))

  0%|          | 0/915 [00:00<?, ?it/s]

In [39]:
translate = { 
    'FreeAgent': 'GoodTestResult', 'Start': 'Stay', 'HR': 'BadTestResult', 
    'Released': 'GoodTestResult', 'Draft': 'GoodTestResult', 'Trade': 'GoodTestResult',
    'Waivers': 'GoodTestResult', 'CndDeal': 'GoodTestResult', 'Purchase': 'GoodTestResult',
    'TopPlayer': 'VitalsCrash'
}

In [40]:
tbl.replace(translate,inplace=True)

In [42]:
tbl

Date           Event
Player                                          
abrej003 0   2013-10-29 00:01:00  GoodTestResult
         1   2014-03-31 12:00:00            Stay
         2   2014-04-02 12:00:00            Stay
         3   2014-04-03 12:00:00            Stay
         4   2014-04-04 12:00:00            Stay
...                          ...             ...
zunim001 718 2019-09-12 12:00:00            Stay
         719 2019-09-15 12:00:00            Stay
         720 2019-09-22 12:00:00            Stay
         721 2019-09-29 12:00:00            Stay
         722 2020-10-30 00:01:00  GoodTestResult

[452995 rows x 2 columns]

## Remove the Off Season
Redefine the dates in the table to remove the gaps created by the offseason.

In [43]:
tbl.rename(columns = { 'Date': 'OrgDate' }, inplace=True)
tbl['DayIdx'] = pd.NaT
tbl['Date'] = pd.NaT

years = np.sort(tbl.OrgDate.dt.year.unique())

mxDay = 0
for year in years:
    iY = tbl.OrgDate.dt.year == year
    startDate = tbl.loc[iY].OrgDate.min()
    days = (tbl.loc[iY].OrgDate - startDate).dt.days
        
    tbl.loc[iY,'DayIdx'] = days + mxDay
    
    mxDay += days.max()
    
tbl['Date'] = tbl.OrgDate.min() + pd.to_timedelta(tbl.DayIdx,'D')
    
tbl = tbl[['Date', 'Event' ]]
tbl


Date           Event
Player                                          
abrej003 0   2009-01-08 00:01:00  GoodTestResult
         1   2009-06-03 00:01:00            Stay
         2   2009-06-05 00:01:00            Stay
         3   2009-06-06 00:01:00            Stay
         4   2009-06-07 00:01:00            Stay
...                          ...             ...
zunim001 718 2014-10-26 00:01:00            Stay
         719 2014-10-29 00:01:00            Stay
         720 2014-11-05 00:01:00            Stay
         721 2014-11-12 00:01:00            Stay
         722 2015-12-11 00:01:00  GoodTestResult

[452995 rows x 2 columns]

## Save Data

In [44]:
baseballDataFileStr = '/Users/gmessier/data/baseball/MLBHospitalData'
tbl.to_hdf(baseballDataFileStr+'.hd5',key='Data',mode='w')
tbl.to_csv(baseballDataFileStr+'.csv')